daily_run estimates strikes and number of contracts for a put selling strategy on equity indices such as S&P 500. 

Algorithm is drawn from  JUREK, J. W. and STAFFORD, E. (2015), The Cost of Capital for Alternative Investments. The Journal of Finance, 70: 2185-2226. doi:10.1111/jofi.12269- 

Requires an open session of either IB Gateway or TWS from Interactive Brokers

In [ ]:
from option_daily_prod import OptionMarket, SpxOptionAsset, RSL2OptionAsset
from spx_data_update import UpdateSP500Data
from option_simulation import OptionSimulation, OptionTrades
import pyfolio as pf
import numpy as np
import pandas as pd
import nest_asyncio
from time import time
import plistlib
file_name = UpdateSP500Data.DATA_BASE_PATH / 'config.plist'
f = open(str(file_name), 'rb')
pl = plistlib.load(f)
illiquid_dict = pl['illiquid_equity']
illiquid_equity = sum(illiquid_dict.values()) * 0.5
illiquid_equity

In [ ]:
before = time()
nest_asyncio.apply()

spx_trades = OptionMarket(SpxOptionAsset())
rsl_trades = OptionMarket(RSL2OptionAsset())

In [ ]:
z_score = np.array([-0.5, -1, -1.5])
leverage = np.array(range(1, 3, 1))
num_expiries = 3

after = time()
spx = True
if spx:
    trd_choice = spx_trades.form_trade_choice(z_score, num_expiries, 'P')
else:
    trd_choice = rsl_trades.form_trade_choice(z_score, num_expiries, 'P')
    
last_price = trd_choice.spot
after_2 = time()
print('elapsed:', np.round(after - before, 1))
print('elapsed:', np.round(after_2 - before, 1))
print(last_price)
print(trd_choice.sigma)
print(trd_choice.account_value)

In [ ]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction:row;
}
"""
HTML('<style>{}</style>'.format(CSS))


In [ ]:
_=[display(df) for df in [trd_choice.strike_grid(), trd_choice.premium_grid(), trd_choice.prices_grid()]]

In [ ]:

prct_decline = trd_choice.pct_otm_grid(last_price)
format_dict = {x: '{:.2%}' for x in prct_decline.to_dict()}
display(prct_decline.style.format(format_dict))

In [ ]:
capital_at_risk = float(trd_choice.account_value[0].value) + illiquid_equity
premium_list = []
premium_list =  [(trd_choice.premium_grid() * contractTable.round() * 100) \
                 for contractTable in trd_choice.option_lots(leverage, capital_at_risk)]

_ = [display(df.style.format('${:,.0f}')) for df in premium_list]  

In [ ]:
premium_list_pct = []
premium_list_pct = [(trd_choice.premium_grid() * lev) / last_price for lev in leverage]
_ = [display(df.style.format('{:.2%}')) for df in premium_list_pct]  

In [ ]:

_ = [display(contractTable.round().style.format('{:.0f}')) for \
     contractTable in trd_choice.option_lots(leverage, capital_at_risk)]
    

In [ ]:
margin_dlr = [trd_choice.margin(last_price) * df for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in margin_dlr]  

In [ ]:
notional_dlr=[df * trd_choice.strike_grid() * 100 for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in notional_dlr]  

In [ ]:
updater = UpdateSP500Data()
opt_sim = OptionSimulation()
dtf_trades, zscore, sim_dates_live = opt_sim.trade_sim(-1, 1, trade_type='THU3')

In [ ]:
opt_trade = OptionTrades(dtf_trades, zscore, sim_dates_live, 2)
returns_geometric, returns_arithmetic = opt_trade.sell_option()
pf.timeseries.aggregate_returns(returns_arithmetic, 'monthly')

In [ ]:
dtf_trades[-1]